# Configuration

Define settings for remote services, create `appsettings.Development.json` using `appsettings.json` as template.

If you want to use this script to insert sample data into your production deployment (after `azd up`) you need to manually give yourself
- `Cognitive Services OpenAI Contributor` rights on the `Azure OpenAI` resource
- `Search Index Data Contributor` right on the `Azure AI Search` resource.

Please execute `azd auth login` after that to have access to these Azure resources.

# Install and Import Required Packages
Install and import the necessary packages using NuGet.

In [5]:
#r "nuget: Azure.Identity, 1.13.1"
#r "nuget: CsvHelper, 33.0.1"
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0"
#r "nuget: Azure.Search.Documents, 11.6.0"
#r "nuget: Microsoft.SemanticKernel, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.33.0-preview"

// Import the necessary libraries
using System;
using System.Threading;
using System.Net;
using System.ComponentModel;
using System.Diagnostics;
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;
using System.Globalization;
using Azure.Identity;
using CsvHelper;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.VectorData;
using Azure.Search.Documents.Indexes;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Embeddings;

Installed Packages Azure.Identity, 1.13.1 Azure.Search.Documents, 11.6.0 CsvHelper, 33.0.1 Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Binder, 9.0.0 Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0 Microsoft.Extensions.Configuration.UserSecrets, 9.0.0 Microsoft.SemanticKernel, 1.33.0 Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.33.0-preview Microsoft.SemanticKernel.Plugins.Core, 1.33.0-alpha

# Import Sample Products

Imports all sample products from `products.csv` and generates for them.

In [12]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0050, SKEXP0001, SKEXP0110, SKEXP0010, OPENAI001

var configBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("./appsettings.json", optional: false)
    .AddJsonFile("./appsettings.Development.json", optional: false);   
var configuration = configBuilder.Build();

internal sealed class ProductDataModel
{
    [VectorStoreRecordKey]
    public string Key { get; set; }

    [VectorStoreRecordData]
    [TextSearchResultName]
    public string Name { get; set; }

    [VectorStoreRecordData]
    [TextSearchResultValue]
    public string Content { get; set; }

    [VectorStoreRecordVector(3072)]
    public ReadOnlyMemory<float> Embedding { get; set; }
}

var collectionName = "products";

var searchIndexClient = new SearchIndexClient(new Uri(configuration["AzureAISearchEndpoint"]), new AzureDeveloperCliCredential());
var vectorStore = new AzureAISearchVectorStore(searchIndexClient);

IVectorStoreRecordCollection<string, ProductDataModel> recordCollection = vectorStore.GetCollection<string, ProductDataModel>(collectionName);
await recordCollection.CreateCollectionIfNotExistsAsync();

var textEmbeddingGeneration = new AzureOpenAITextEmbeddingGenerationService(
    configuration["EmbeddingModelDeployment"],
    configuration["AzureOpenAIEndpoint"],
    new AzureDeveloperCliCredential());


using (var reader = new StreamReader("./products.csv"))
using (var csv = new CsvReader(reader, CultureInfo.InvariantCulture))
{
    var records = csv.GetRecords<dynamic>().ToList();

    foreach (var record in records)
    {
        var product = new ProductDataModel
        {
            Key = Guid.NewGuid().ToString(),
            Name = record.Name,
            Content = record.Content,
            Embedding = await textEmbeddingGeneration.GenerateEmbeddingAsync((string)record.Name + "" + (string)record.Content)
        };
        await recordCollection.UpsertAsync(product);
    }
}